# Quality Control for Tissue-Compartment Models

## Setup

In [ ]:
# system functions that are always useful to have
import time, sys, os
from pprint import pprint

# basic numeric setup
import nibabel as nib
import numpy as np
import pickle

from Boxcar import Boxcar
from RadialArtery import RadialArtery
from Raichle1983Model import Raichle1983Model
from Mintun1984Model import Mintun1984Model
from Huang1980Model import Huang1980Model
from Ichise2002Model import Ichise2002Model

### <font color='orange'>_High-level definitions below configure QC to follow_</font>

In [ ]:
# define filesystems, and high-level identifiers
sub = "sub-108306"
ses = "ses-20230227112148"
trc = "trc-oo"
petdir = os.path.join(os.getenv("SINGULARITY_HOME"), "CCIR_01211", "derivatives", sub, ses, "pet")
assert os.path.isdir(petdir), f"{petdir} not found"

kern = os.path.join(os.getenv("SINGULARITY_HOME"), "CCIR_01211", "sourcedata", "kernel_hct=41.1.nii.gz")

timeAppend = "timeAppend-4"
MODEL = "Mintun1984"

parc_index = 2  # csf ~ 2,3,10,11,15,21,22
#parc_index = 6  # thalamus ~ 6, 25
#parc_index = 1  # deep white ~ 1, 20
#parc_index = 110  # cortex ~ 110 - 309

tag = "main7-rc1p85-3000"

In [ ]:
# define data files
idif_0 = os.path.join(petdir, f"{sub}_{ses}_{trc}_proc-MipIdif_idif.nii.gz")
twil_0 = os.path.join(petdir, f"{sub}_{ses}_{trc}_proc-TwiliteKit-do-make-input-func-nomodel_inputfunc.nii.gz")
assert os.path.isfile(idif_0), f"{idif_0} not found"
assert os.path.isfile(twil_0), f"{twil_0} not found"
assert os.path.isfile(kern), f"{kern} not found"

idif = os.path.join(petdir, f"{sub}_{ses}_{trc}_proc-MipIdif_idif_dynesty-Boxcar-ideal.nii.gz")
twil = os.path.join(petdir, f"{sub}_{ses}_{trc}_proc-TwiliteKit-do-make-input-func-nomodel_inputfunc_dynesty-RadialArtery-ideal.nii.gz")
assert os.path.isfile(idif), f"{idif} not found"
assert os.path.isfile(twil), f"{twil} not found"

pet = os.path.join(petdir, f"{sub}_{ses}_{trc}_proc-delay0-BrainMoCo2-createNiftiMovingAvgFrames_{timeAppend}-ParcSchaeffer-reshape-to-schaeffer-schaeffer.nii.gz")
assert os.path.isfile(pet), f"{pet} not found"

model_fp = f"{sub}_{ses}_{trc}_proc-delay0-BrainMoCo2-createNiftiMovingAvgFrames_{timeAppend}-schaeffer-{MODEL}"
qm_idif = os.path.join(petdir, f"{model_fp}Boxcar-{tag}-qm.nii.gz")
qm_twil = os.path.join(petdir, f"{model_fp}Artery-{tag}-qm.nii.gz")
assert os.path.isfile(qm_idif), f"{qm_idif} not found"
assert os.path.isfile(qm_twil), f"{qm_twil} not found"

pickle_idif = os.path.join(petdir, f"{model_fp}Boxcar-{tag}-res.pickle")
pickle_twil = os.path.join(petdir, f"{model_fp}Artery-{tag}-res.pickle")
assert os.path.isfile(pickle_idif), f"{pickle_idif} not found"
assert os.path.isfile(pickle_twil), f"{pickle_twil} not found"

# define models

nii_idif = nib.load(qm_idif)
img_idif = nii_idif.get_fdata()
nii_twil = nib.load(qm_twil)
img_twil = nii_twil.get_fdata()

In [ ]:
truths_idif = img_idif[parc_index,]
truths_twil = img_twil[parc_index,]
model_cls = globals()[MODEL+"Model"]
mm_idif = model_cls(
    idif, 
    pet,
    truths=truths_idif,
    nlive=30)
mm_twil = model_cls(
    twil, 
    pet,
    truths=truths_twil,
    nlive=30)

## Inspect Mintun1984Model twil

In [ ]:
v = mm_twil.truths
pprint("===================================== v ======================================")
pprint(v)

data = mm_twil.data(v)
pprint("==================================== data ====================================")
pprint(data)

pprint("================================== mm_twil.martin_v1_measurement ===================================")
pprint(mm_twil.martin_v1_measurement)

pprint("================================== mm_twil.raichle_ks_measurement ===================================")
pprint(mm_twil.raichle_ks_measurement)

## Plot Mintun1984Model twil

In [ ]:
mm_twil.plot_truths(parc_index=parc_index)

In [ ]:
mm_twil.plot_variations(0, 0.14, 0.74, mm_twil.truths)

In [ ]:
with open(pickle_twil, 'rb') as f:
    package = pickle.load(f)

Let's sample from this distribution using the default `dynesty` settings with `'slice'`.

In [ ]:
# de novo
# res_twil = mm_twil.run_nested_for_indexed_tac(parc_index, print_progress=True)

# # restart
# sampler = dynesty.DynamicNestedSampler.restore(mm_twil.fqfp+"_dynesty-Boxcar-20240122210738.save")
# sampler.run_nested(resume=True, print_progress=True)
# res = sampler.results

## Inspect Mintun1984Model idif

In [ ]:
v = mm_idif.truths
pprint("===================================== v ======================================")
pprint(v)

data = mm_idif.data(v)
pprint("==================================== data ====================================")
pprint(data)

pprint("================================== mm_twil.martin_v1_measurement ===================================")
pprint(mm_idif.martin_v1_measurement)

pprint("================================== mm_twil.raichle_ks_measurement ===================================")
pprint(mm_idif.raichle_ks_measurement)

## Plot Mintun1984Model idif

In [ ]:
mm_idif.plot_truths(parc_index=parc_index)

In [ ]:
mm_idif.plot_variations(0, 0.14, 0.74, mm_idif.truths)

In [ ]:
with open(pickle_idif, 'rb') as f:
    package = pickle.load(f)

Let's sample from this distribution using the default `dynesty` settings with `'slice'`.

In [ ]:
# de novo
# res_idif = mm_idif.run_nested_for_indexed_tac(parc_index, print_progress=True)

# # restart
# sampler = dynesty.DynamicNestedSampler.restore(mm_idif.fqfp+"_dynesty-Boxcar-20240122210738.save")
# sampler.run_nested(resume=True, print_progress=True)
# res = sampler.results